In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('../input/c/digit-recognizer/train.csv')
# test_df = pd.read_csv('../input/c/digit-recognizer/test.csv')

In [ ]:
trainX = train_df.loc[:, train_df.columns != 'label'] 
trainY = train_df['label']

trainX, validX, trainY, validY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)

trainX = np.array(trainX).reshape((trainX.shape[0], 28, 28, 1))
trainX = trainX.astype('float32')
trainX = trainX / 255.0

validX = np.array(validX).reshape((validX.shape[0], 28, 28, 1))
validX = validX.astype('float32')
validX = validX / 255.0

trainY = keras.utils.to_categorical(trainY)
validY = keras.utils.to_categorical(validY)

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(validX.shape)
print(validY.shape)

In [ ]:
model = keras.Sequential()

model.add(layers.Conv2D(32, (7, 7), activation='relu', padding="same", kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(32, (7, 7), activation='relu', padding="same", kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(32, (7, 7), activation='relu', padding="same", kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))
# model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (7, 7), activation='relu', padding="same", kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))
# model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (7, 7), activation='relu', padding="same", kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))
# model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (7, 7), activation='relu', padding="same", kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))
# model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Flatten())

NUMBER_OF_HIDDEN_LAYERS = 5

for i in range(NUMBER_OF_HIDDEN_LAYERS):
    model.add(layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

model.add(layers.Dense(10, activation='softmax'))
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=20,
    min_delta=0.001,
    restore_best_weights=True,
)
history = model.fit(
    trainX, trainY,
    validation_data=(validX, validY),
    batch_size=512,
    epochs=200,
    callbacks=[early_stopping],
)

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Cross-entropy")
history_df.loc[:, ['accuracy', 'val_accuracy']].plot(title="Accuracy")

In [ ]:
history_df.head(1000)

In [ ]:
test_df = pd.read_csv('../input/c/digit-recognizer/test.csv')

In [ ]:
testX = np.array(test_df).reshape((test_df.shape[0], 28, 28, 1))

testX = testX.astype('float32')
testX = testX / 255.0

In [ ]:
predictions = model.predict(testX)

In [ ]:
image_idx = np.random.randint(0, testX.shape[0])
print(np.argmax(predictions[image_idx]))
plt.imshow(testX[image_idx])

In [ ]:
print(len(predictions))

In [ ]:
submission = pd.DataFrame(columns=['ImageId','Label'])
for i in range(len(predictions)):
    submission = submission.append({'ImageId': i+1, 'Label': np.argmax(predictions[i])}, ignore_index=True)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('./submission.csv', index=False)